In [1]:
import numpy as np

# Imports

In [2]:
%load_ext filprofiler

In [3]:
import json
import numpy
import pandas as pd
import pickle
import sys

In [4]:
sys.path.append('../src/')

In [5]:
import prepare_data

# Configs

In [6]:
settings_path = '../configs/settings.json'
with open(settings_path, 'r') as f:
    config = json.load(f)

In [7]:
# EDA 3)
with open(f"../{config['RAW_DATA_DIR']}/index_stock_weights.pkl", 
          'rb') as f:
    index_stock_weights = pickle.load(f)

# Create training clear 

## Read

In [8]:
%%time
df_raw = pd.read_csv(f"../{config['RAW_DATA_DIR']}/train.csv")

CPU times: user 8.37 s, sys: 606 ms, total: 8.98 s
Wall time: 8.98 s


## Scale sizes

In [9]:
train = df_raw
size_col = ['imbalance_size','matched_size','bid_size','ask_size']

### Dict for inference

In [10]:
scale_dict = {} # feature -> stock_id -> median
for col in size_col:
    medians = train.groupby('stock_id')[col].median()
    scale_dict[col] = medians.to_dict()

In [11]:
with open(f"../{config['RAW_DATA_DIR']}/scale_dict.pkl", 'wb') as f:
    pickle.dump(scale_dict, f)

In [12]:
%%time

import importlib
importlib.reload(prepare_data)

df, feas_list = prepare_data.generate_features_no_hist_polars(df_raw, 
                                                   index_stock_weights,
                                                   scale_dict)

1
2
3
4
5
6
1 is done
3 is done
12 is done
6 is done
7
8
CPU times: user 3min 14s, sys: 38.7 s, total: 3min 53s
Wall time: 1min 41s


In [13]:
print(f'{df.shape=}')
print(f'{feas_list=}')

df.shape=(5237980, 406)
feas_list=['imb1_wap_mid_price', 'imb1_ask_money_bid_money', 'imb1_volume_cont_volumn_auc', 'imb1_reference_price_ask_price', 'imb1_reference_price_mid_price', 'seconds_in_bucket', 'div_flag_imbalance_size_2_balance', 'ask_price', 'imb1_reference_price_bid_price', 'scale_matched_size', 'imb1_near_price_wap', 'volumn_auc_money', 'imb1_far_price_wap', 'bid_size', 'scale_bid_size', 'bid_size_all', 'rolling18_mean_imb1_auc_ask_size_auc_bid_size', 'rolling3_mean_div_flag_imbalance_size_2_balance', 'rolling60_std_div_flag_imbalance_size_2_balance', 'rolling36_mean_flag_imbalance_size', 'rolling3_std_imb1_auc_ask_size_auc_bid_size', 'rolling18_mean_imb1_ask_size_all_bid_size_all', 'rolling6_mean_div_flag_imbalance_size_2_balance', 'rolling6_std_imb1_auc_ask_size_auc_bid_size', 'rolling3_mean_imb1_auc_ask_size_auc_bid_size', 'rolling60_std_imb1_auc_ask_size_auc_bid_size', 'rolling6_std_bid_size_all', 'rolling3_std_bid_size_all', 'rolling3_mean_bid_size_all', 'rolling18_

### Scale without dict

In [ ]:
for _ in size_col:
    train[f"scale_{_}"] = train[_] / train.groupby(['stock_id'])[_].transform('median')

In [ ]:
df.shape